<a href="https://colab.research.google.com/github/mainarel/Numerical_Methods_6/blob/main/2_Heat_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np

aa = 1

a0, b0, a1, b1 = -1, 1, 1, 1

def u0(x,t):
  return np.cosh(x-t)

def f(x,t):
  return - np.sinh(x-t) - np.cosh(x-t)

#начальные условия
def phi(x):
  return np.cosh(x)

#граничные условия
def gamma1(t):
  return np.exp(t)

def gamma2(t):
  return np.exp(1-t)

xmin =  0
xmax = 1
h_cur = 0.01

tmin = 0
tmax = 1
tau_cur = 0.005

xk_cur = np.arange(xmin, xmax + h_cur, h_cur)
tn_cur = np.arange(tmin, tmax, tau_cur)

In [ ]:
def thomas_algorithm(a,b,c,f, n):
    A = np.zeros(n)
    B = np.zeros(n)
    y = np.zeros(n)

    A[0] = -c[0] / b[0]
    B[0] = f[0] / b[0]

    for i in range(1, n - 1):
        A[i] = -c[i] / (b[i] + a[i] * A[i - 1])
    A[-1] = 0
    for i in range(1, n):
        B[i] = (f[i] - a[i] * B[i - 1]) / (b[i] + a[i] * A[i - 1])

    y[-1] = B[-1]
    for i in reversed(range(n-1)):
        y[i] = B[i] + A[i] * y[i + 1]
    return y

def p1(prev, tau, sigma, h):
    xk = np.arange(xmin, xmax+h, h)
    tn = np.arange(tmin, tmax, tau)
    
    a = np.zeros(len(xk))
    b = np.zeros(len(xk))
    c = np.zeros(len(xk))
    d = np.zeros(len(xk))

    d[0] = gamma1(tn*tau)
    d[-1] = gamma2(tn*tau)

    b[0] = -(a0/h) + b0
    c[0] = a0/h

    b[-1] = (a1 / h) + b1
    a[-1] = -a1/h

    for i in range(1, len(xk)-1):
        a[i] = tau * aa**2* sigma / h**2
    for i in range(1, len(xk)-1):
        b[i] = -1 - 2 * tau * aa**2 * sigma/ h**2
    for i in range(1, len(xk)-1):
        c[i] = tau * aa**2* sigma / h**2
    for i in range(1, len(xk)-1):
        d[i] = - prev[i] -tau * f(xk[i], (tn - 0.5)*tau) +(sigma-1)*(tau * aa**2 / h ** 2)*(prev[i+1] - 2*prev[i] + prev[i-1])

    return thomas_algorithm(a,b ,c ,d ,len(xk))


def p2(prev, tau, sigma, h):

    xk = np.arange(xmin, xmax+h, h)
    tn = np.arange(tmin, tmax, tau)

    a = np.zeros(len(xk))
    b = np.zeros(len(xk))
    c = np.zeros(len(xk))
    d = np.zeros(len(xk))

    if a0 == 0:
        b[0] = b0
        d[0] = gamma1(tn*tau)
    else:
        b[0] = 1 - aa ** 2 * tau / (h ** 2 * 2) * (-2 + b0 * 2 * h / a0)
        c[0] = - aa ** 2 * tau / h ** 2
        d[0] = prev[0]+aa**2*tau/(2*h**2)*(-gamma1(tn*tau)*2*h/a0 + prev[1] - 2*prev[0]+prev[1] - (gamma1((tn-1)*tau)-b0*prev[0])*2*h/a0) + tau*f(xk[0], (tn - 0.5)*tau)

    if a1 == 0:
        b[-1] = b1
        d[-1] = gamma2(tn*tau)
    else:
        d[-1] = prev[0]+aa**2*tau/(2*h**2)*(gamma2(tn*tau)*2*h/a1 + prev[-2] - 2*prev[-1]+prev[-2] + (gamma2((tn-1)*tau)-b1*prev[0])*2*h/a1) + tau*f(xk[-1], (tn - 0.5)*tau)
        b[-1] = 1 - aa**2*tau/(h**2*2)*(-2 - b1*2*h/a1)
        a[-1] = - np.arange**2*tau/h**2

    for i in range(1, len(xk) - 1):
        a[i] = tau * aa ** 2 * sigma / h ** 2
    for i in range(1, len(xk) - 1):
        b[i] = -1 - 2 * tau * aa ** 2 * sigma / h ** 2
    for i in range(1, len(xk) - 1):
        c[i] = tau * aa ** 2 * sigma / h ** 2
    for i in range(1, len(xk) - 1):
        d[i] = - prev[i] - tau * f(xk[i], (tn - 0.5) * tau) + (sigma - 1) * (tau * aa ** 2 / h ** 2) * (prev[i + 1] - 2 * prev[i] + prev[i - 1])

    return thomas_algorithm(a, b, c, d, len(xk))